In [2]:
cd /glade/work/joyceyang

/glade/work/joyceyang


In [3]:
# Imports 
%matplotlib inline
import os, glob
import xarray as xr
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd  
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import datetime 
from datetime import date, datetime, timedelta

lat_grid = np.load('lat_grid.npy')
lon_grid = np.load('lon_grid.npy')

In [30]:
def nansumwrapper(a, **kwargs):
    if np.isnan(a).all():
        return np.nan
    else:
        return np.nansum(a, **kwargs)

c = np.arange(12, dtype=np.float_).reshape(4,3)
c[1:4, 0:3] = np.nan

nansumwrapper(c, axis=1)

array([3., 0., 0., 0.])

In [29]:
c

array([[ 0.,  1.,  2.],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

In [31]:
a = np.array([np.nan, np.nan])
nansumwrapper(a, axis=0)

nan

In [2]:
ixy = xr.DataArray([1,1,1,2,2,2,3,3,3,4,4,4], dims='pft')
lon = xr.DataArray([0,0,0,110,110,110,212,212,212,358,358,358], dims='pft')
jxy = xr.DataArray([1,1,1,1,2,2,2,2,3,3,3,3], dims='pft')
lat = xr.DataArray([-90,-90,-90,-90,10,10,10,10,84,84,84,84], dims='pft')
active = xr.DataArray([1,1,0,1,0,0,0,0,1,0,1,1], dims='pft')
wtlunit = xr.DataArray([0,0.1,0,0,0.2,0.3,1,0,0,0.1,0.2,0], dims='pft')
itype_lunit = xr.DataArray([1,1,2,7,4,5,6,7,8,9,2,4], dims='pft')
tsa = xr.DataArray([3,7,9,0,12,4,5,6,8,1,2,90], dims='pft')

In [10]:
tsa.value.max()

<bound method ImplementsArrayReduce._reduce_method.<locals>.wrapped_func of <xarray.DataArray (pft: 12)>
array([ 3,  7,  9,  0, 12,  4,  5,  6,  8,  1,  2, 90])
Dimensions without coordinates: pft>

In [9]:
# to turn into numpy arrays
#lon = lon.values
#ixy = ixy.values

<xarray.DataArray (pft: 12)>
array([1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4])
Dimensions without coordinates: pft

In [10]:
# pull out values (prolly don't use this one cause it doesn't retain the shape and size of the array )
lon[(ixy==2)&(jxy==1)].size

1

In [11]:
# mask values (preferred for now)
lon.where(ixy==2)

<xarray.DataArray (pft: 12)>
array([ nan,  nan,  nan, 110., 110., 110.,  nan,  nan,  nan,  nan,  nan,  nan])
Dimensions without coordinates: pft

In [7]:
# count unique values per lat/lon pair 
max_length = 0
for i in range(1,5):
    for j in range(1,4):
        print(i,j)
        print(lon.where(ixy==i).where(jxy==j).values)
        new_length = tsa.where(ixy==i).where(jxy==j).dropna(dim='pft').values.size
        if new_length > max_length:
            max_length = new_length

print('Max number of unique values per unique lat/lon pair =',max_length)

1 1
[ 0.  0.  0. nan nan nan nan nan nan nan nan nan]
1 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
1 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
2 1
[ nan  nan  nan 110.  nan  nan  nan  nan  nan  nan  nan  nan]
2 2
[ nan  nan  nan  nan 110. 110.  nan  nan  nan  nan  nan  nan]
2 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
3 1
[nan nan nan nan nan nan nan nan nan nan nan nan]
3 2
[ nan  nan  nan  nan  nan  nan 212. 212.  nan  nan  nan  nan]
3 3
[ nan  nan  nan  nan  nan  nan  nan  nan 212.  nan  nan  nan]
4 1
[nan nan nan nan nan nan nan nan nan nan nan nan]
4 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
4 3
[ nan  nan  nan  nan  nan  nan  nan  nan  nan 358. 358. 358.]
Max number of unique values per unique lat/lon pair = 3


In [8]:
# find max pft for each unique lat/lon/lunit combo
max_length = 0
tsa = tsa.where(itype_lunit==1) # only look at longitude values where the lunit type is 1 aka veg
# ^ this is the first prerequisite 

# now loop through each latitude/longitude combo 
for i in range(1,5):
    for j in range(1,4):
        print(i,j)
        print(tsa.where(ixy==i).where(jxy==j).values)
        new_length = tsa.where(ixy==i).where(jxy==j).dropna(dim='pft').values.size
        if new_length > max_length:
            max_length = new_length

print('Max unique values per lat/lon pair =', max_length)

1 1
[ 3.  7. nan nan nan nan nan nan nan nan nan nan]
1 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
1 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
2 1
[nan nan nan nan nan nan nan nan nan nan nan nan]
2 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
2 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
3 1
[nan nan nan nan nan nan nan nan nan nan nan nan]
3 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
3 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
4 1
[nan nan nan nan nan nan nan nan nan nan nan nan]
4 2
[nan nan nan nan nan nan nan nan nan nan nan nan]
4 3
[nan nan nan nan nan nan nan nan nan nan nan nan]
Max unique values per lat/lon pair = 2


In [ ]:
# Game Plan 
# Go through and create list of tsa values for each unique lat/lon pair for the land unit of interest 
# Then initialize a cube with dimensions of lat x lon x max pft (for that land unit?)
# Loop through the tsa list and fill in each value one by one into the previously created cube 

In [4]:
cube1 = np.full((4,3,3), float("NaN"))

In [5]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(figsize=(20,10))
ax = plt.axes(projection=ccrs.PlateCarree()) 
ax.add_feature(cfeature.BORDERS, linestyle=':') 
ax.set_global()
ax.coastlines()
plt.tight_layout()

tsa_grid_veg = np.nansum((np.multiply(tsa_cube_veg, tsa_wtlunit_cube_veg)), axis=2) # find weighted mean TSA for each grid cell by multiplying TSA cube and pft weight cube and summing by grid cell
c = tsa_grid_veg - 273 # convert to Celsius 
np.save('tsa_grid_veg', tsa_grid_veg) 

plt.scatter(lon_grid, lat_grid, c=c, transform=ccrs.PlateCarree(), s=10, marker="s") # create scatter plot with longtitude for the x dimension, latitude as the y dimension, and TSA as the z dimension

plt.title("Vegetation Temperature - January 2015", size=20);
cbar = plt.colorbar(shrink=0.6) # create colorbar 
cbar.set_label('Temperature (C)    ', rotation=270)

plt.savefig('Vegetation_Jan2015.jpg')